# Capella API: Search, Order, and Download Open Data Europe Region

In [ ]:
# Required libraries:
# requests
# json
# urllib

Your username and password must be saved in a .json file named 'credentials.json' and formatted as follows.

{"username": "yourusername","password": "xxxxxxxxx"}

### Set up Project Variables

In [ ]:
data_collection = ["capella-open-data"]
# Europe AOI
aoi = {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -33.046875,
              83.7539108491127
            ],
            [
              -53.4375,
              83.40004205976699
            ],
            [
              -64.6875,
              81.72318761821155
            ],
            [
              -73.828125,
              79.03843742487174
            ],
            [
              -71.3671875,
              75.23066741281573
            ],
            [
              -55.54687499999999,
              58.99531118795094
            ],
            [
              -12.65625,
              36.31512514748051
            ],
            [
              24.609375,
              34.88593094075317
            ],
            [
              150.46875,
              42.293564192170095
            ],
            [
              190.546875,
              62.103882522897855
            ],
            [
              187.3828125,
              72.71190310803662
            ],
            [
              100.8984375,
              81.46626086056541
            ],
            [
              -14.0625,
              83.4803661137381
            ],
            [
              -33.046875,
              83.7539108491127
            ]
          ]
        ]
      }

### Import required libraries, build a print utility function, assign API endpoints and load Credentials

In [ ]:
import requests
import json

# JSON utility function
def p(data):
    print(json.dumps(data, indent=2))

# Capella API endpoints
URL = 'https://api.capellaspace.com'
token = '/token'
collections = '/catalog/collections'
catsearch = '/catalog/search'
orders = '/orders/'

#Load username and password
with open('credentials.json') as f:
    data = json.load(f)
    username = data['username']
    password = data['password']

### Get and Print Access Token

In [ ]:
#Get the token
r = requests.post(URL + token, 
                  headers = {'Content-Type': 'application/x-www-form-urlencoded'}, auth=(username,password))

accesstoken = r.json()["accessToken"]

# Print the token
#print("Access Token: " + accesstoken)

headers = {'Authorization':'Bearer ' + accesstoken}

### Print Available Collections

In [ ]:
# See what collections are available
r = requests.get(URL + collections, headers=headers)

# Print the results
#p(r.json())

### Post Search Filters, Print the Results

In [ ]:
# Post search filters
filters = {
  #"bbox": [-180,-90,180,90], # lower left coodinate and upper right coordinate, in decimal degrees
  "intersects": aoi,
  "limit": 1000, # overwrite the default pagination limit of 10, adjust as necessary
  "collections": data_collection, #["capella-open-data"], # specify the desired collection  "sentinel-s1-l2"
  "sortby": "properties.datetime"
}

headers = {'Content-Type': 'application/json',
  'Accept': 'application/geo+json', 'Authorization':'Bearer ' + accesstoken}
r = requests.post(URL + catsearch, json=filters, headers=headers)

# Inspect the results
#p(r.json())

### Make and Post an Order

In [ ]:
# Make an Order
features = r.json()["features"]
granulelist = []

# Loop over all the features from the response and add to an array for an order
for f in features:
    item = {"CollectionId": f["collection"], "GranuleId": f["id"]}
    granulelist.append(item)

cnt = len(features)
print(cnt)
myorder = {"Items": granulelist}
# Post the order and inspect the result
r = requests.post(URL + orders, json=myorder, headers=headers)
#p(r.json())

### Get the STAC records with the signed URLs using the /download endpoint, Print the Result

In [ ]:
myorderid = r.json()["orderId"]
r = requests.get(URL + orders + myorderid + '/download', headers=headers)
#p(r.json())

### Download the Results

In [ ]:
features = r.json()
     
basefp = 'C:/data/open_data/' # Local directory to save data

for feature in features:
           
    filepath = feature["assets"]["HH"]["href"] # the second nested dictionary ("HH" here) must be changed for different assets
    # e.g. filepath = feature["assets"]["metadata"]["href"] will return the url for the metadata file

    filename = filepath[filepath.rfind("/")+1:]
    sep = "?"
    truncname = filename.split(sep, 1)[0]
    outfp = basefp + truncname

    import urllib
    f = urllib.request.urlretrieve(filepath, outfp)
    with requests.get(filepath, stream=True) as result:
        result.raise_for_status()
        with open(outfp, 'wb') as f:
            for chunk in result.iter_content(chunk_size=10000000):
                f.write(chunk)      